# Cours d'optimisation réseau pour le MVA
# Problème de placement de machines virtuelles
Paul CHAUVIN \\
paulchauvin97@gmail.com

# Environment

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading data

In [3]:
path = '/content/drive/MyDrive/Cours_MVA/2nd semestre/optimisation réseau/Data.xlsx'
df = pd.read_excel(path)
small_df = df.sample(n=100)
print(df.head(n=10))
print(" ")
print(small_df)

   vCPU  Memory (GB)  Disk Space (GB)  Class
0     4          8.0            10.23      1
1     8         16.0            10.23      1
2     2          4.0            32.00      1
3     2          4.0            32.00      1
4     4          8.0            10.23      1
5     4          8.0            10.23      1
6    32         64.0          1019.02      1
7    32         64.0          1019.02      1
8    32         64.0          1019.02      1
9     8         16.0            10.23      1
 
     vCPU  Memory (GB)  Disk Space (GB)  Class
511     2          4.0            10.23      3
834     2          4.0            10.23      3
55      4          8.0            10.23      2
695     2          2.0            40.00      3
259    16         32.0           519.02      2
..    ...          ...              ...    ...
241     2          4.0            50.00      2
773     2          4.0            10.23      3
205     1          1.0            50.00      2
110     2          4.0           

# Define the server's capacity

In [4]:
#define the serveur components :
chosen_dataframe = df.sample(frac=1)
vCPU_s = 64
memory_s = 512
disk_space_s = 2048

# First method: brute force 

In [5]:
# initialization
number_of_serveur = 1
current_vCPU_s = vCPU_s
current_memory_s = memory_s
current_disk_space_s = disk_space_s
impossible_VM = pd.DataFrame({'vCPU': [0], 'Memory (GB)': [0], 'Disk Space (GB)': [0]})

# check if problem is feasible
if (max(chosen_dataframe['vCPU'])>vCPU_s or max(chosen_dataframe['Memory (GB)'])>memory_s or max(chosen_dataframe['Disk Space (GB)'])>disk_space_s):
  print('Impossible')

#predict number of servers if problem feasible
for index, row in chosen_dataframe.iterrows():
  if (row['vCPU']>vCPU_s or row['Memory (GB)']>memory_s or row['Disk Space (GB)']>disk_space_s):
    impossible_VM = impossible_VM.append(pd.DataFrame({'vCPU': [0] + row['vCPU'], 'Memory (GB)': [0] + row['Memory (GB)'], 'Disk Space (GB)': [0] + row['Disk Space (GB)']}), ignore_index=True)

  elif (row['vCPU'] <= current_vCPU_s and row['Memory (GB)'] <= current_memory_s and row['Disk Space (GB)'] <= current_disk_space_s):
    current_vCPU_s = current_vCPU_s - row['vCPU']
    current_memory_s = current_memory_s - row['Memory (GB)']
    current_disk_space_s = current_disk_space_s - row['Disk Space (GB)']
    
  else:
    number_of_serveur = number_of_serveur + 1
    current_vCPU_s = vCPU_s - row['vCPU']
    current_memory_s = memory_s - row['Memory (GB)']
    current_disk_space_s = disk_space_s - row['Disk Space (GB)']

print('Number of servers needed if possible: ',number_of_serveur)
print('Impossible VM: ', impossible_VM.tail(-1))

Impossible
Number of servers needed if possible:  91
Impossible VM:     vCPU  Memory (GB)  Disk Space (GB)
1   2.0          8.0           4096.0
2   4.0         32.0           3152.0
3  16.0         32.0           2074.0
4   2.0          8.0           4096.0
5   2.0          8.0           4096.0
6   2.0          8.0           4096.0


# Second method: keep in memory the uncomplete servers

In [6]:
# initialization
df_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
new_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
impossible_VM = pd.DataFrame({'vCPU': [0], 'Memory (GB)': [0], 'Disk Space (GB)': [0]})

# check if problem is feasible
if (max(chosen_dataframe['vCPU'])>vCPU_s or max(chosen_dataframe['Memory (GB)'])>memory_s or max(chosen_dataframe['Disk Space (GB)'])>disk_space_s):
  print('Impossible')

#predict number of servers if problem feasible
for index, row in chosen_dataframe.iterrows():
  if (row['vCPU']>vCPU_s or row['Memory (GB)']>memory_s or row['Disk Space (GB)']>disk_space_s):
    impossible_VM = impossible_VM.append(pd.DataFrame({'vCPU': [0] + row['vCPU'], 'Memory (GB)': [0] + row['Memory (GB)'], 'Disk Space (GB)': [0] + row['Disk Space (GB)']}), ignore_index=True)  
    continue

  #print(row)
  for index, server in df_server.iterrows() :
    stopping = 0
    if (row['vCPU'] <= server['vCPU'] and row['Memory (GB)'] <= server['Memory (GB)']  and row['Disk Space (GB)'] <= server['Disk Space (GB)']):
        server['vCPU'] = server['vCPU'] - row['vCPU']
        server['Memory (GB)'] = server['Memory (GB)']  - row['Memory (GB)']
        server['Disk Space (GB)'] = server['Disk Space (GB)'] - row['Disk Space (GB)']
        stopping = 1
        break
  if (stopping == 0):
    df_server = df_server.append(pd.DataFrame({'vCPU': [vCPU_s] - row['vCPU'], 'Memory (GB)': [memory_s] - row['Memory (GB)'], 'Disk Space (GB)': [disk_space_s] - row['Disk Space (GB)']}), ignore_index=True)
  #print(df_server.head())


print('Number of servers needed: ',df_server.shape[0])
print('Impossible VM: ', impossible_VM.tail(-1))

Impossible
Number of servers needed:  77
Impossible VM:     vCPU  Memory (GB)  Disk Space (GB)
1   2.0          8.0           4096.0
2   4.0         32.0           3152.0
3  16.0         32.0           2074.0
4   2.0          8.0           4096.0
5   2.0          8.0           4096.0
6   2.0          8.0           4096.0


# Third method: ordering on the most difficult constraint

In [7]:
# initialization
df_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
new_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
impossible_VM = pd.DataFrame({'vCPU': [0], 'Memory (GB)': [0], 'Disk Space (GB)': [0]})


#get most difficult constraint
percent_vCPU = 100 - 100*((- chosen_dataframe['vCPU'] + vCPU_s)/vCPU_s).mean()
percent_memory_s = 100 - 100*((- chosen_dataframe['Memory (GB)'] + memory_s)/memory_s).mean()
percent_disk_space_s = 100 - 100*((- chosen_dataframe['Disk Space (GB)'] + disk_space_s)/disk_space_s).mean()

hardest_constraint = max(percent_vCPU, percent_memory_s, percent_disk_space_s)
second_hardest_constraint = sorted([percent_vCPU, percent_memory_s, percent_disk_space_s])[-2]
easiest_constraint = min(percent_vCPU, percent_memory_s, percent_disk_space_s)

if percent_vCPU==hardest_constraint:
  constraint1 = 'vCPU'

if percent_vCPU==second_hardest_constraint:
  constraint2 = 'vCPU'

if percent_vCPU==easiest_constraint:
  constraint3 = 'vCPU'

if percent_memory_s==hardest_constraint:
  constraint1 = 'Memory (GB)'

if percent_memory_s==second_hardest_constraint:
  constraint2 = 'Memory (GB)'

if percent_memory_s==easiest_constraint:
  constraint3 = 'Memory (GB)'

if percent_disk_space_s==hardest_constraint:
  constraint1 = 'Disk Space (GB)'

if percent_disk_space_s==second_hardest_constraint:
  constraint2 = 'Disk Space (GB)'

if percent_disk_space_s==easiest_constraint:
  constraint3 = 'Disk Space (GB)'


# check if problem is feasible
if (max(chosen_dataframe['vCPU'])>vCPU_s or max(chosen_dataframe['Memory (GB)'])>memory_s or max(chosen_dataframe['Disk Space (GB)'])>disk_space_s):
  print('Impossible')

#order the VMs based on the hardest constraints
chosen_dataframe_ordered = chosen_dataframe.sort_values(by=[constraint3, constraint2, constraint1], ascending=[False, False, False])

#predict number of servers if problem feasible
for index, row in chosen_dataframe_ordered.iterrows():
  if (row['vCPU']>vCPU_s or row['Memory (GB)']>memory_s or row['Disk Space (GB)']>disk_space_s):
    impossible_VM = impossible_VM.append(pd.DataFrame({'vCPU': [0] + row['vCPU'], 'Memory (GB)': [0] + row['Memory (GB)'], 'Disk Space (GB)': [0] + row['Disk Space (GB)']}), ignore_index=True)  
    continue

  #df_server_ordered = df_server_ordered.sort_values(by=[constraint1, constraint2, constraint3], ascending=[False, False, False])
  for index, server in df_server.iterrows() :
    stopping = 0
    if (row['vCPU'] <= server['vCPU'] and row['Memory (GB)'] <= server['Memory (GB)']  and row['Disk Space (GB)'] <= server['Disk Space (GB)']):
        server['vCPU'] = server['vCPU'] - row['vCPU']
        server['Memory (GB)'] = server['Memory (GB)']  - row['Memory (GB)']
        server['Disk Space (GB)'] = server['Disk Space (GB)'] - row['Disk Space (GB)']
        stopping = 1
        break
  if (stopping == 0):
    df_server = df_server.append(pd.DataFrame({'vCPU': [vCPU_s] - row['vCPU'], 'Memory (GB)': [memory_s] - row['Memory (GB)'], 'Disk Space (GB)': [disk_space_s] - row['Disk Space (GB)']}), ignore_index=True)
  #print(df_server.head())


print('Number of servers needed: ',df_server.shape[0])
print('Impossible VM: ', impossible_VM.tail(-1))

Impossible
Number of servers needed:  81
Impossible VM:     vCPU  Memory (GB)  Disk Space (GB)
1  16.0         32.0           2074.0
2   4.0         32.0           3152.0
3   2.0          8.0           4096.0
4   2.0          8.0           4096.0
5   2.0          8.0           4096.0
6   2.0          8.0           4096.0


# Fourth method: ordering randomly and keeping best solution


In [8]:
# initialization
best_prediction = chosen_dataframe.shape[0]

# check if problem is feasible
if (max(chosen_dataframe['vCPU'])>vCPU_s or max(chosen_dataframe['Memory (GB)'])>memory_s or max(chosen_dataframe['Disk Space (GB)'])>disk_space_s):
  print('Impossible')


for random in range(100):
  df_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
  new_server = pd.DataFrame({'vCPU': [vCPU_s], 'Memory (GB)': [memory_s], 'Disk Space (GB)': [disk_space_s]})
  impossible_VM = pd.DataFrame({'vCPU': [0], 'Memory (GB)': [0], 'Disk Space (GB)': [0]})

  #predict number of servers if problem feasible
  random_dataframe = chosen_dataframe.sample(frac=1)

  for index, row in random_dataframe.iterrows():
    if (row['vCPU']>vCPU_s or row['Memory (GB)']>memory_s or row['Disk Space (GB)']>disk_space_s):
      impossible_VM = impossible_VM.append(pd.DataFrame({'vCPU': [0] + row['vCPU'], 'Memory (GB)': [0] + row['Memory (GB)'], 'Disk Space (GB)': [0] + row['Disk Space (GB)']}), ignore_index=True)  
      continue

    #print(row)
    for index, server in df_server.iterrows() :
      stopping = 0
      if (row['vCPU'] <= server['vCPU'] and row['Memory (GB)'] <= server['Memory (GB)']  and row['Disk Space (GB)'] <= server['Disk Space (GB)']):
          server['vCPU'] = server['vCPU'] - row['vCPU']
          server['Memory (GB)'] = server['Memory (GB)']  - row['Memory (GB)']
          server['Disk Space (GB)'] = server['Disk Space (GB)'] - row['Disk Space (GB)']
          stopping = 1
          break
    if (stopping == 0):
      df_server = df_server.append(pd.DataFrame({'vCPU': [vCPU_s] - row['vCPU'], 'Memory (GB)': [memory_s] - row['Memory (GB)'], 'Disk Space (GB)': [disk_space_s] - row['Disk Space (GB)']}), ignore_index=True)
    
  prediction_servers = df_server.shape[0]

  if prediction_servers < best_prediction : 
    best_prediction = prediction_servers
    
print('Number of servers needed, best case: ', best_prediction)

Impossible
Number of servers needed, best case:  75


# Other cases 
- partially loaded: we should have a list of servers and their capacities. Generally, we try to load first the ones having less available space.
- splitted VMs: when a VMs is above a certain limit, we split the VM, taking the same ratio for the 2 other dimensions. And we try to put the VM in the servers already existing, ordering them by the biggest values first. The algorithm opens a new server when one of the three dimension is full in all the servers. 
- VMs criticity: you first try to put all ones together, all three togethers. and then you complete with twos where we can put them, ordering them by size. 
- Anti-affinity: pretty close to criticity levels
- The proposed algorithms cannot be online. 